In [1]:
from smoothie.auth.AuthorizationHandler import AuthorizationHandler
from smoothie.data_objects.User import User
import aiohttp
import urllib
import getpass

In [2]:
session = aiohttp.ClientSession()
auth_handler = AuthorizationHandler(session)

Since Spotify requires a login page with user interaction, we'll need to perform this first step in a browser page. Copy and paste the result of the next cell into a browser and approve access to your Spotify account.

In [3]:
print(f'https://accounts.spotify.com/authorize?client_id={auth_handler.spotify_client_id}&response_type=code&redirect_uri={urllib.parse.quote_plus(auth_handler._redirect_uri)}')

https://accounts.spotify.com/authorize?client_id=c6571c2bc3f444a18ad0308ab4b712f3&response_type=code&redirect_uri=https%3A%2F%2Fgoogle.com


Now, in the resulting `google.com` url, copy all the text after `code=`, and enter it when prompted in the next cell

In [5]:
code = getpass.getpass(prompt='Code from URL? ')
me = User(code=code)

Get access/refresh tokens for use with future API calls

In [6]:
await auth_handler.request_access_token(me)

{'access_token': 'BQCR9f72QA3woDo9ojH-2UCrKm81b146ioQ0VtF-7VyUmaV4-vMKvpfNccK0LNFQSKDjwd5CNqGAbM8VuTd7v-tamZ9v8tI76fxOGv_HebXMkQyKRgLABy2airCLAdPqULLH5jkztfiJ4RaXwQXFpvJoirbFgWdEaQ', 'token_type': 'Bearer', 'expires_in': 3600, 'refresh_token': 'AQB9WBn-Bj636UfWWyXkq7IWEpXeqzJyKZdTtarTv-nfSj2ri5O7IBeb13pR4kUThmgWA-bG_S5b043CgVLZaMyV9ql7d9ZC4MQTswCuRps0WRhecRZqkIf6ctyeNneWMOA'}


In [12]:
import dataclasses
print(dataclasses.asdict(me))

{'code': 'AQDXxjATVgHNjA7LvL53is4Q4K5GHjTn-l-ySe6ntkYsaU_n_luG10K09xGnVUtkzVYv6HobwTDfX6VEgoxzKDZLHAEue4gnoSqfCmxBlxc_AkbPCNm16jptXRehefprGrDudQkD4MDndwwvqWAjR9wuQi6cRg', 'access_token': 'BQCR9f72QA3woDo9ojH-2UCrKm81b146ioQ0VtF-7VyUmaV4-vMKvpfNccK0LNFQSKDjwd5CNqGAbM8VuTd7v-tamZ9v8tI76fxOGv_HebXMkQyKRgLABy2airCLAdPqULLH5jkztfiJ4RaXwQXFpvJoirbFgWdEaQ', 'refresh_token': 'AQB9WBn-Bj636UfWWyXkq7IWEpXeqzJyKZdTtarTv-nfSj2ri5O7IBeb13pR4kUThmgWA-bG_S5b043CgVLZaMyV9ql7d9ZC4MQTswCuRps0WRhecRZqkIf6ctyeNneWMOA', 'user_id': None, 'token_expiration_time': None}
